In [ ]:
%pylab inline

import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [ ]:
#Getting the MNIST data provided by Tensorflow
from tensorflow.examples.tutorials.mnist import input_data

#Loading in the mnist data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images,\
    mnist.test.labels

In [ ]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

In [ ]:
# helper functions
def dense_to_one_hot(labels_dense, num_classes=10):
    #"""Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

In [ ]:
### set all variables

# number of neurons in each layer
input_num_units = 800
hidden_num_units = 500
output_num_units = 10

# define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])

# set remaining variables
epochs = 10
batch_size = 128
learning_rate = 0.01

### define weights and biases of the neural network (refer this article if you don't understand the terminologies)

weights = {
    'hidden': tf.Variable(tf.random_normal([input_num_units, hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.random_normal([hidden_num_units, output_num_units], seed=seed))
}

biases = {
    'hidden': tf.Variable(tf.random_normal([hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.random_normal([output_num_units], seed=seed))
}

In [ ]:
hidden_layer = tf.add(tf.matmul(x, weights['hidden']), biases['hidden'])
hidden_layer = tf.nn.relu(hidden_layer)

output_layer = tf.matmul(hidden_layer, weights['output']) + biases['output']

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output_layer,labels= y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
# load dataset from DBN model
train=load('rbm_dbn_troutput.npy')
train_x=train
test=load('rbm_dbn_teoutput.npy')
test_x=test


In [ ]:
for i in range(1): 
    with tf.Session() as sess:
        # create initialized variable
        sess.run(init)
    
        ### for each epoch, do:
        ###   for each batch, do:
        ###     create pre-processed batch
        ###     run optimizer by feeding batch
        ###     find cost and reiterate to minimize
    
        for epoch in range(epochs):
            avg_cost = 0
            total_batch = int(train_x.shape[0]/batch_size)
            for start, end in zip(range(0, len(train_x), batch_size), range(batch_size, len(train_x), batch_size)):
                #batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
                batch_x = train_x[start:end]
                y_val = trY[start:end]
                batch_y = dense_to_one_hot(y_val)
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
            
                avg_cost += c / total_batch
            
            print ("Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost))
    
        print("\nTraining complete!")
    
    
    # find predictions on val set
   # pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
   # accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
   # print( "Validation Accuracy:", accuracy.eval({x: train_x.reshape(-1, input_num_units), y: dense_to_one_hot(trY)}))
    
        predict = tf.argmax(output_layer, 1)
        pred = predict.eval({x: test_x.reshape(-1, input_num_units)})
    

In [ ]:
# Accuracy of DBN

print(np.sum(pred==teY)/10000)

In [ ]:
# load dataset from DAN model
train=load('rbm_dan_troutput.npy')
train_x=train
test=load('rbm_dan_teoutput.npy')
test_x=test

In [ ]:
for i in range(1): 
    with tf.Session() as sess:
        # create initialized variable
        sess.run(init)
    
        ### for each epoch, do:
        ###   for each batch, do:
        ###     create pre-processed batch
        ###     run optimizer by feeding batch
        ###     find cost and reiterate to minimize
    
        for epoch in range(epochs):
            avg_cost = 0
            total_batch = int(train_x.shape[0]/batch_size)
            for start, end in zip(range(0, len(train_x), batch_size), range(batch_size, len(train_x), batch_size)):
                #batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
                batch_x = train_x[start:end]
                y_val = trY[start:end]
                batch_y = dense_to_one_hot(y_val)
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
            
                avg_cost += c / total_batch
            
            print ("Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost))
    
        print("\nTraining complete!")
    
    
    # find predictions on val set
   # pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
   # accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
   # print( "Validation Accuracy:", accuracy.eval({x: train_x.reshape(-1, input_num_units), y: dense_to_one_hot(trY)}))
    
        predict = tf.argmax(output_layer, 1)
        pred = predict.eval({x: test_x.reshape(-1, input_num_units)})
    

In [ ]:
# Accuracy of DAN
print(np.sum(pred==teY)/10000)